In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


[Learn the Basics](intro.html) ||
**Quickstart** ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
[Build Model](buildmodel_tutorial.html) ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Quickstart
This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

## Working with data
PyTorch has two [primitives to work with data](https://pytorch.org/docs/stable/data.html):
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html),
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like
CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element
in the dataloader iterable will return a batch of 64 features and labels.



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](data_tutorial.html).




--------------




In [4]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
help(mlflow.MlflowClient().create_experiment)

Help on method create_experiment in module mlflow.tracking.client:

create_experiment(name: str, artifact_location: Optional[str] = None, tags: Optional[Dict[str, Any]] = None) -> str method of mlflow.tracking.client.MlflowClient instance
    Create an experiment.
    
    :param name: The experiment name. Must be unique.
    :param artifact_location: The location to store run artifacts.
                              If not provided, the server picks an appropriate default.
    :param tags: A dictionary of key-value pairs that are converted into
                            :py:class:`mlflow.entities.ExperimentTag` objects, set as
                            experiment tags upon experiment creation.
    :return: String as an integer ID of the created experiment.
    
    .. code-block:: python
        :caption: Example
    
        from pathlib import Path
        from mlflow import MlflowClient
    
        # Create an experiment with a name that is unique and case sensitive.
        c

In [6]:
# client = mlflow.MlflowClient(tracking_uri="http://127.0.0.1:5000")
# experiment_description = (
#     "This is the tutorial of pytorch. "
# )

# experiment_tags = {
#     "project_name": "Fashion_MNIST",
#     "mlflow.note.content": experiment_description,
# }

# MNIST_experiment = client.create_experiment(name="DNN", tags=experiment_tags)


In [5]:
# Sets the current active experiment to the "Apple_Models" experiment and returns the Experiment metadata
apple_experiment = mlflow.set_experiment("DNN")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "MNIST_test"

# Define an artifact path that the model will be saved to.
artifact_path = "MNIST"

In [17]:
mlflow.pytorch.autolog()

In [12]:
help(mlflow.pytorch.autolog)

Help on function autolog in module mlflow.pytorch:

autolog(log_every_n_epoch=1, log_every_n_step=None, log_models=True, log_datasets=True, disable=False, exclusive=False, disable_for_unsupported_versions=False, silent=False, registered_model_name=None, extra_tags=None)
    .. Note:: Autologging is known to be compatible with the following package versions: ``1.6.0`` <= ``torch`` <= ``2.0.1``. Autologging may not succeed when used with package versions outside of this range.
    
    
    Enables (or disables) and configures autologging from `PyTorch Lightning
    <https://pytorch-lightning.readthedocs.io/en/latest>`_ to MLflow.
    
    Autologging is performed when you call the `fit` method of
    `pytorch_lightning.Trainer()     <https://pytorch-lightning.readthedocs.io/en/latest/trainer.html#>`_.
    
    Explore the complete `PyTorch MNIST     <https://github.com/mlflow/mlflow/tree/master/examples/pytorch/MNIST>`_ for
    an expansive example with implementation of additional ligh

## Creating Models
To define a neural network in PyTorch, we create a class that inherits
from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network
in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate
operations in the neural network, we move it to the GPU or MPS if available.



In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
import os

import lightning as L
import torch
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.cli import LightningCLI
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics.functional import accuracy
from torchvision import datasets, transforms

import mlflow.pytorch

class LightningMNISTClassifier(L.LightningModule):
    def __init__(self, learning_rate=0.01):
        """
        Initializes the network
        """
        super().__init__()

        # mnist images are (1, 28, 28) (channels, width, height)
        self.optimizer = None
        self.scheduler = None
        self.layer_1 = torch.nn.Linear(28 * 28, 512)
        self.layer_2 = torch.nn.Linear(512, 512)
        self.layer_3 = torch.nn.Linear(512, 10)
        self.learning_rate = learning_rate
        self.val_outputs = []
        self.test_outputs = []

    def forward(self, x):
        """
        :param x: Input data

        :return: output - mnist digit label for the input image
        """
        batch_size = x.size()[0]

        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)

        # layer 1 (b, 1*28*28) -> (b, 512)
        x = self.layer_1(x)
        x = torch.relu(x)

        # layer 2 (b, 512) -> (b, 512)
        x = self.layer_2(x)
        x = torch.relu(x)

        # layer 3 (b, 512) -> (b, 10)
        x = self.layer_3(x)

        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)

        return x

    def cross_entropy_loss(self, logits, labels):
        """
        Initializes the loss function

        :return: output - Initialized cross entropy loss function
        """
        return F.nll_loss(logits, labels)

    def training_step(self, train_batch, batch_idx):
        """
        Training the data as batches and returns training loss on each batch

        :param train_batch: Batch data
        :param batch_idx: Batch indices

        :return: output - Training loss
        """
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {"loss": loss}

    def validation_step(self, val_batch, batch_idx):
        """
        Performs validation of data in batches

        :param val_batch: Batch data
        :param batch_idx: Batch indices

        :return: output - valid step loss
        """
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        self.val_outputs.append(loss)
        return {"val_step_loss": loss}

    def on_validation_epoch_end(self):
        """
        Computes average validation loss
        """
        avg_loss = torch.stack(self.val_outputs).mean()
        self.log("val_loss", avg_loss, sync_dist=True)
        self.val_outputs.clear()

    def test_step(self, test_batch, batch_idx):
        """
        Performs test and computes the accuracy of the model

        :param test_batch: Batch data
        :param batch_idx: Batch indices

        :return: output - Testing accuracy
        """
        x, y = test_batch
        output = self.forward(x)
        _, y_hat = torch.max(output, dim=1)
        test_acc = accuracy(y_hat.cpu(), y.cpu(), task="multiclass", num_classes=10)
        self.test_outputs.append(test_acc)
        return {"test_acc": test_acc}

    def on_test_epoch_end(self):
        """
        Computes average test accuracy score
        """
        avg_test_acc = torch.stack(self.test_outputs).mean()
        self.log("avg_test_acc", avg_test_acc, sync_dist=True)
        self.test_outputs.clear()

    def configure_optimizers(self):
        """
        Initializes the optimizer and learning rate scheduler

        :return: output - Initialized optimizer and scheduler
        """
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        self.scheduler = {
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                mode="min",
                factor=0.2,
                patience=2,
                min_lr=1e-6,
                verbose=True,
            ),
            "monitor": "val_loss",
        }
        return [self.optimizer], [self.scheduler]


In [ ]:
import pytorch_lightning as pl
class NNModule(pl.LightningModule):
    def __init__(self, ) -> None:
        super().__init__()

    def forward(self, imgs):

    def configure_optimizers(self):

    def training_step(self, batch, batch_idx):

    def validation_step(self, batch, batch_idx):

    def test_step(self, batch, batch_idx):

Read more about [building neural networks in PyTorch](buildmodel_tutorial.html).




--------------




## Optimizing the Model Parameters
To train a model, we need a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions)
and an [optimizer](https://pytorch.org/docs/stable/optim.html).



In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and
backpropagates the prediction error to adjust the model's parameters.



In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.



In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [8]:
torch.cuda.is_available()

True

In [9]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    #with mlflow.start_run() as run:
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 2.301971  [   64/60000]
loss: 2.291928  [ 6464/60000]
loss: 2.269500  [12864/60000]
loss: 2.263815  [19264/60000]
loss: 2.256734  [25664/60000]
loss: 2.227207  [32064/60000]
loss: 2.231507  [38464/60000]
loss: 2.199894  [44864/60000]
loss: 2.191018  [51264/60000]
loss: 2.166434  [57664/60000]
Test Error: 
 Accuracy: 40.6%, Avg loss: 2.158883 

Epoch 2
-------------------------------
loss: 2.169904  [   64/60000]
loss: 2.160477  [ 6464/60000]
loss: 2.103833  [12864/60000]
loss: 2.113905  [19264/60000]
loss: 2.073478  [25664/60000]
loss: 2.019391  [32064/60000]
loss: 2.036571  [38464/60000]
loss: 1.963902  [44864/60000]
loss: 1.957310  [51264/60000]
loss: 1.895320  [57664/60000]
Test Error: 
 Accuracy: 57.8%, Avg loss: 1.888790 

Epoch 3
-------------------------------
loss: 1.928461  [   64/60000]
loss: 1.893884  [ 6464/60000]
loss: 1.778176  [12864/60000]
loss: 1.807499  [19264/60000]
loss: 1.709059  [25664/60000]
loss: 1.665848  [32064/600

Read more about [Training your model](optimization_tutorial.html).




--------------




## Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [22]:
help(torch.save)

Help on function save in module torch.serialization:

save(obj: object, f: Union[str, os.PathLike, BinaryIO, IO[bytes]], pickle_module: Any = <module 'pickle' from '/home/jy_zhang/anaconda3/envs/pytorch/lib/python3.11/pickle.py'>, pickle_protocol: int = 2, _use_new_zipfile_serialization: bool = True, _disable_byteorder_record: bool = False) -> None
    save(obj, f, pickle_module=pickle, pickle_protocol=DEFAULT_PROTOCOL, _use_new_zipfile_serialization=True)
    
    Saves an object to a disk file.
    
    See also: :ref:`saving-loading-tensors`
    
    Args:
        obj: saved object
        f: a file-like object (has to implement write and flush) or a string or
           os.PathLike object containing a file name
        pickle_module: module used for pickling metadata and objects
        pickle_protocol: can be specified to override the default protocol
    
    .. note::
        A common PyTorch convention is to save tensors using .pt file extension.
    
    .. note::
        PyTo

In [12]:
torch.save(model.state_dict(), "model2.pth",_use_new_zipfile_serialization = False)

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [14]:
data = torch.load("model.pth")
data

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0064,  0.0315, -0.0321,  ..., -0.0101,  0.0273,  0.0081],
                      [ 0.0022,  0.0187, -0.0348,  ..., -0.0086,  0.0019,  0.0317],
                      [-0.0080,  0.0343, -0.0164,  ...,  0.0348, -0.0317,  0.0242],
                      ...,
                      [-0.0227,  0.0337, -0.0025,  ..., -0.0222, -0.0263, -0.0269],
                      [ 0.0132,  0.0280,  0.0144,  ..., -0.0317,  0.0196,  0.0077],
                      [-0.0294,  0.0313, -0.0134,  ...,  0.0300, -0.0182, -0.0079]],
                     device='cuda:0')),
             ('linear_relu_stack.0.bias',
              tensor([-1.9945e-02, -2.9500e-02,  1.6078e-02, -2.8489e-02, -7.7808e-04,
                       1.2074e-02, -7.4671e-03, -2.6765e-02,  4.3378e-04,  2.8306e-02,
                       1.8506e-02, -4.8354e-03,  4.3320e-03,  1.9686e-02,  1.8991e-02,
                       1.4672e-02, -1.5688e-02, -2.6641e-02,  2.3393e-02,  1.4413

In [15]:
import pickle
f = open('model2.pth','rb')
data = pickle.load(f)
data

119547037146038801333356

## Loading Models

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it.



In [26]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.



In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Read more about [Saving & Loading your model](saveloadrun_tutorial.html).


